# TO DO:
- email subject? 
- send email function
- session variables, integrate with flask

In [15]:
from dotenv import load_dotenv
import os
import json
load_dotenv()
H2O_API_KEY = os.getenv("H2O_API_KEY") 
from h2ogpte import H2OGPTE

In [16]:
# SESSION VARIABLES (part 1 / 2)
# session["emails"], session["lang_pref"] =  ...,...
emails = ["hannahnga@gmail.com",]
lang_pref = ["English","Spanish", "Mandarin"]

In [17]:
def loading(data_dir):
    client = H2OGPTE(
        address='https://h2ogpte.genai.h2o.ai',
        api_key= H2O_API_KEY,
    )

    # Create a new collection
    collection_id = client.create_collection(
        name='devMeetingAssistant',
        description='Test development for meeting assistant (prompt engineering)',
    )
    with open(data_dir, 'rb') as f:
        meeting_data1 = client.upload('Data/fake_sample.txt', f)
    client.ingest_uploads(collection_id, [meeting_data1,])
    chat_session_id = client.create_chat_session(collection_id)
    # in flask, return add client, collection_id to session variables
    #  session['collection_id'] = collection_id
    # 
    return client, collection_id

In [18]:
def rag_chat(client, chat_session_id, main_prompt, system_prompt):
    with client.connect(chat_session_id) as session:
        answer = session.query(
            message=main_prompt,
            system_prompt=system_prompt,
            rag_config={
            "rag_type": "rag", # https://h2oai.github.io/h2ogpte/getting_started.html#advanced-controls-for-document-q-a
            },
            llm='h2oai/h2ogpt-4096-llama2-70b-chat',
        )
        return answer

def extract_json_string(full_string): 
    """Extracts json-like string from LLM response. Outputs str."""

    start_index = full_string.find('{')  # Find the index of the first '{'
    end_index = full_string.rfind('}')   # Find the index of the last '}'
    
    if start_index != -1 and end_index != -1:
        return full_string[start_index:end_index + 1]
    else:
        return None
    
def string_like_JSON_to_txt(json_str):
    """Input: json-like string
    output: Email body string"""
    if json_str is None:
        return "Invalid JSON format. Cannot parse the response."

    # Parse the extracted JSON string
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        return "Error parsing JSON data."

    # Extract information from the parsed data
    agenda = data.get('Agenda', '')
    meeting_summary = data.get('Meeting Summary', '')
    actionables = data.get('Actionables', [])

    # Format the post-meeting email
    email_subject = f"Post-Meeting Summary: {agenda}"
    email_body = f"Dear Team,\n\nHere's a summary of our recent meeting:\n\nAgenda: {agenda}\n\nMeeting Summary:\n{meeting_summary}\n\nActionables:\n"
    for action in actionables:
        email_body += f"- {action['Action']} (Assigned to: {action['Assigned']}, Deadline: {action['Deadline']}, Priority: {action['Priority']})\n"

    email_body += "\nPlease let me know if anything needs clarification or if there are additional action items to add.\n\nBest regards,\n[Your Name]"
    return email_body

In [28]:
# Prompts
def get_summary_response(client, chat_session_id):
    system_prompt = """
    You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    main_prompt = """
    Execute all the following steps: \
    1. Examine the document and identify all names of the attendees, remember these names for the next steps.
    2. Identify and extract the main purpose or agenda of the meeting
    3. Identify all actionables mentioned during the meeting
    4. Extract all the actionables and action plans, identify all details related to it including the deadlines, who it is assigned to, etc.
    5. Summarize the entire transcript
    6. Present all identified information in the following JSON format for easy parsing:
    {"Agenda": "", "Meeting Summary":"", "Actionables":[{"Action":"", "Deadline":"", "Assigned":""}, ...]}
    DO NOT include anything other than the JSON format output in your response.
    """
    return rag_chat(client, chat_session_id, main_prompt, system_prompt)

def get_sentiment_response(client, chat_session_id, meeting_summary_response):
    system_prompt = """
    You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    sentiment_prompt = f"""
    Execute all the following steps: \
    1. Examine the contexts of this JSON object: {meeting_summary_response}
    2. For each actionable, execute the following steps
    a. identify the sentence in the document where it is mentioned
    b. Interpret the sentence, based on this context, categorize the priority of this task as "High", "Medium" or "Low"
    c. Edit the JSON object, add a column to the dictionary for this action as follows: "Priority": ""
    3. Present all identified information in the following JSON format:
    {{"Agenda": "", "Meeting Summary":""}} for the entire meeting
    {{"Actionables":[{{"Action":"", "Deadline":"", "Assigned":"", "Priority":""}}]}} for each action plan
    """

    return rag_chat(client, chat_session_id, sentiment_prompt, system_prompt)

def get_language_translation(client, chat_session_id, message):
    emails_dict = {"English": message}
    system_prompt = """
    You are a Multilingual Meeting Assistant for a large company that wants to maximize meeting efficiency for their global staff. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    for lang in lang_pref:
        if lang not in emails_dict:
            translate_prompt = f"""
            Execute all the following steps: \
            1. Examine the contexts of this email body written in English, denoted in triple backticks: {message}
            2. Translate the entire email into {lang}, maintain the same tone and formatting.
            3. Respond with the translated email text ONLY.
            4. You do not need to include "(Translated by Google Translate) at the end of the email output"
            5. You do not need to output the original email.
            """

            translated_email = rag_chat(client, chat_session_id, translate_prompt, system_prompt)
            emails_dict[lang] = translated_email.content
            # print(translated_email.content)

    return emails_dict


In [20]:
def get_LLM_response(data_dir):
    """
    Output: json-like string
    """
    
    client, collection_id = loading(data_dir)
    chat_session_id = client.create_chat_session(collection_id)
    # 1. Summary and action plan
    meeting_summary_response = get_summary_response(client, chat_session_id)

    # 2. Sentiment analysis/Priority allocation
    sentiment_reply = get_sentiment_response(client, chat_session_id ,extract_json_string(meeting_summary_response.content))
    # print(sentiment_reply.content)
    # Return as JSON-like string
    return extract_json_string(sentiment_reply.content), client, chat_session_id

In [21]:
# pip install h2ogpte==1.4.10

In [22]:
# More session variables
data_dir = "fake_sample.txt"
json_str, client, chat_session_id = get_LLM_response(data_dir)
show_email = string_like_JSON_to_txt(json_str) # to display to user

In [23]:
print(json_str)

{
"Agenda": "Reviewing progress on the project, discussing roadblocks, and planning next steps",
"Meeting Summary": "The meeting discussed the current status of the project, reviewed the design proposal, and finalized the budget allocation. The team also discussed potential roadblocks and found solutions, and scheduled a follow-up discussion. The meeting concluded with a summary of actionables and their deadlines.",
"Actionables": [
{
"Action": "Complete design proposal",
"Deadline": "Wednesday",
"Assigned": "Ryan_Edward",
"Priority": "High"
},
{
"Action": "Confirm meeting with design specialist",
"Deadline": "Thursday",
"Assigned": "Ben_CH",
"Priority": "Medium"
},
{
"Action": "Resolve integration issue",
"Deadline": "Friday",
"Assigned": "Chan Yi Ru Micole",
"Priority": "Medium"
},
{
"Action": "Share progress document",
"Deadline": "After the meeting",
"Assigned": "Ryan_Edward",
"Priority": "Low"
},
{
"Action": "Send out meeting details",
"Deadline": "Thursday",
"Assigned": "Ben_CH",

In [24]:
# After user confirms email, run translation
def confirm_responses(client, chat_session_id, message):
    # get dictionary of responses in different languages
    responses = get_language_translation(client, chat_session_id, message)

    # send email
    return responses

In [29]:
responses = confirm_responses(client, chat_session_id, show_email)

In [31]:
for k,v in responses.items():
    print("")
    print("Language:",k)
    print("")
    print(v)


Language: English

Dear Team,

Here's a summary of our recent meeting:

Agenda: Reviewing progress on the project, discussing roadblocks, and planning next steps

Meeting Summary:
The meeting discussed the current status of the project, reviewed the design proposal, and finalized the budget allocation. The team also discussed potential roadblocks and found solutions, and scheduled a follow-up discussion. The meeting concluded with a summary of actionables and their deadlines.

Actionables:
- Complete design proposal (Assigned to: Ryan_Edward, Deadline: Wednesday, Priority: High)
- Confirm meeting with design specialist (Assigned to: Ben_CH, Deadline: Thursday, Priority: Medium)
- Resolve integration issue (Assigned to: Chan Yi Ru Micole, Deadline: Friday, Priority: Medium)
- Share progress document (Assigned to: Ryan_Edward, Deadline: After the meeting, Priority: Low)
- Send out meeting details (Assigned to: Ben_CH, Deadline: Thursday, Priority: Low)

Please let me know if anything ne